# Artists + Artworks CSVs

## Setup

Run the cell below to import necessary modules. 

In [128]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

Now, copy the paths to the life expectancy dataset and census dataset to the `le_path` and `census_path` variables respectively in the cell below. Then, run the cell to save your variables.  

In [129]:
import pandas as pd
moma_art_path = "./data/moma_artworks.csv"
moma_art = pd.read_csv(moma_art_path)

print(moma_art.head())

moma_artists_path = "./data/moma_artists.csv"
moma_artists = pd.read_csv(moma_artists_path)

print(moma_artists.head())

met_all_path = "./data/met_artworks_artists.csv"
met_all = pd.read_csv(met_all_path)

print(met_all.head())

   Artwork ID                                              Title Artist ID                      Name  Date  \
0           2  Ferdinandsbrücke Project, Vienna, Austria, Ele...      6210               Otto Wagner  1896   
1           3  City of Music, National Superior Conservatory ...      7470  Christian de Portzamparc  1987   
2           4  Villa near Vienna Project, Outside Vienna, Aus...      7605                Emil Hoppe  1903   
3           5  The Manhattan Transcripts Project, New York, N...      7056           Bernard Tschumi  1980   
4           6  Villa, project, outside Vienna, Austria, Exter...      7605                Emil Hoppe  1903   

                                              Medium                          Dimensions Acquisition Date  \
0      Ink and cut-and-pasted painted pages on paper  19 1/8 x 66 1/2" (48.6 x 168.9 cm)       1996-04-09   
1                  Paint and colored pencil on print       16 x 11 3/4" (40.6 x 29.8 cm)       1995-01-17   
2  Graphite,

/var/folders/k5/983zh5mj7j5fljhdvdr9kjlw0000gn/T/ipykernel_16392/2094333094.py:13: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  met_all = pd.read_csv(met_all_path)


  Object Number  Is Highlight  Is Public Domain  Object ID                Department Object Name  \
0    1979.486.1         False             False          1  American Decorative Arts        Coin   
1    1980.264.5         False             False          2  American Decorative Arts        Coin   
2      67.265.9         False             False          3  American Decorative Arts        Coin   
3     67.265.10         False             False          4  American Decorative Arts        Coin   
4     67.265.11         False             False          5  American Decorative Arts        Coin   

                          Title Culture Period Dynasty Reign Portfolio Artist Role Artist Prefix  \
0  One-dollar Liberty Head Coin     NaN    NaN     NaN   NaN       NaN       Maker           NaN   
1  Ten-dollar Liberty Head Coin     NaN    NaN     NaN   NaN       NaN       Maker           NaN   
2    Two-and-a-Half Dollar Coin     NaN    NaN     NaN   NaN       NaN         NaN           NaN   

## split met_all into met_artworks and met_artists

In [130]:
# column names in met_all mapped to new column names

met_artists_raw = met_all[[
    "Artist Display Name",
    "Artist Nationality",
    "Artist Begin Date",
    "Artist End Date"
]].copy()

#drop columns where name is null
met_artists = met_artists_raw.dropna(subset=["Artist Display Name"]).reset_index(drop=True)
#pick duplicates
met_artists = met_artists.drop_duplicates(subset=["Artist Display Name"]).reset_index(drop=True)
#replace null nationality wth "Unknown Nationality" 
met_artists["Artist Nationality"] = met_artists["Artist Nationality"].fillna("Unknown Nationality")
#begin and end dates to numbers
met_artists["Artist Begin Date"] = pd.to_numeric(met_artists["Artist Begin Date"], errors='coerce')
met_artists["Artist End Date"] = pd.to_numeric(met_artists["Artist End Date"], errors='coerce')
#replace null begin date with -1
met_artists["Artist Begin Date"] = met_artists["Artist Begin Date"].fillna(-1)
#replace null end date with -1
met_artists["Artist End Date"] = met_artists["Artist End Date"].fillna(-1)
#add artist id column to artists
met_artists.insert(0, "ArtistID", range(1, 1 + len(met_artists)))

print(met_artists.head())

#print number of null columns
print(met_artists.isnull().sum())

   ArtistID       Artist Display Name   Artist Nationality  Artist Begin Date  Artist End Date
0         1     James Barton Longacre             American             1794.0           1869.0
1         2        Christian Gobrecht  Unknown Nationality             1785.0           1844.0
2         3           Bela Lyon Pratt  Unknown Nationality             1867.0           1917.0
3         4            Mexican Artist  Unknown Nationality               -1.0             -1.0
4         5  James Gillinder and Sons             American             1861.0           1930.0
ArtistID               0
Artist Display Name    0
Artist Nationality     0
Artist Begin Date      0
Artist End Date        0
dtype: int64


In [131]:
#isolate met artworks columns
# from this:
# 	ArtworkID INT PRIMARY KEY, 
# 	Title VARCHAR(50), 
# 	Medium VARCHAR(50), 
# 	YearStart YEAR, 
# 	YearEnd YEAR, 
# 	Nationality VARCHAR(50), 
# 	Image VARCHAR(100), 
# 	Description VARCHAR(500),
# 	Museum VARCHAR(50), 
# 	Artist VARCHAR(50),
# 	FOREIGN KEY Museum REFERENCES Museum(MuseumID),
# 	FOREIGN KEY Artist REFERENCES Artist(ArtistID)
# FOREIGN KEY HistoricalEvent REFERENCES HistoricalEvents(ArtistID)

met_artworks = met_all[[
    "Object ID",
    "Title",
    "Medium",
    "Object Begin Date",
    "Object End Date",
    "Artist Nationality",
    "Artist Display Name"
]].copy()

#drop columns where title is null
met_artworks = met_artworks.dropna(subset=["Title"]).reset_index(drop=True)
#replace null medium with "Unknown Medium"
met_artworks["Medium"] = met_artworks["Medium"].fillna("Unknown Medium")
#replace null artist / artist nationality
met_artworks["Artist Display Name"] = met_artworks["Artist Display Name"].fillna("Unknown Artist")
met_artworks["Artist Nationality"] = met_artworks["Artist Nationality"].fillna("Unknown Nationality")
#begin and end dates to numbers
met_artworks["Object Begin Date"] = pd.to_numeric(met_artworks["Object Begin Date"], errors='coerce')
met_artworks["Object End Date"] = pd.to_numeric(met_artworks["Object End Date"], errors='coerce')
#replace null begin date with -1
met_artworks["Object Begin Date"] = met_artworks["Object Begin Date"].fillna(-1)
#replace null end date with -1
met_artworks["Object End Date"] = met_artworks["Object End Date"].fillna(-1)

#add artist id column to artworks by mapping from met_artists
artist_id_map = dict(zip(met_artists["Artist Display Name"], met_artists["ArtistID"]))
met_artworks["ArtistID"] = met_artworks["Artist Display Name"].map(artist_id_map)
#if artist is unknown, set artist id to -1
met_artworks["ArtistID"] = met_artworks["ArtistID"].fillna(-1).astype(int)

print(met_artworks.head())

#print number of null columns
print(met_artworks.isnull().sum())

   Object ID                         Title Medium  Object Begin Date  Object End Date   Artist Nationality  \
0          1  One-dollar Liberty Head Coin   Gold               1853             1853             American   
1          2  Ten-dollar Liberty Head Coin   Gold               1901             1901  Unknown Nationality   
2          3    Two-and-a-Half Dollar Coin   Gold               1909             1927  Unknown Nationality   
3          4    Two-and-a-Half Dollar Coin   Gold               1909             1927  Unknown Nationality   
4          5    Two-and-a-Half Dollar Coin   Gold               1909             1927  Unknown Nationality   

     Artist Display Name  ArtistID  
0  James Barton Longacre         1  
1     Christian Gobrecht         2  
2         Unknown Artist      1858  
3         Unknown Artist      1858  
4         Unknown Artist      1858  
Object ID              0
Title                  0
Medium                 0
Object Begin Date      0
Object End Date  

In [132]:
# rename columns
met_artists = met_artists.rename(columns={
    "Artist Display Name": "Name",
    "Artist Nationality": "Nationality",
    "Artist Begin Date": "BirthYear",
    "Artist End Date": "DeathYear"
})

met_artworks = met_artworks.rename(columns={
    "Object ID": "ArtworkID",
    "Title": "Title",
    "Medium": "Medium",
    "Object Begin Date": "YearStart",
    "Object End Date": "YearEnd",
    "Artist Nationality": "Nationality",
    "Artist Display Name": "Artist",
})

#add museum column with "The Metropolitan Museum of Art"
met_artworks["Museum"] = "The Metropolitan Museum of Art"

print(met_artists.head())
print(met_artworks.head())
print(met_artists.isnull().sum())
print(met_artworks.isnull().sum())

   ArtistID                      Name          Nationality  BirthYear  DeathYear
0         1     James Barton Longacre             American     1794.0     1869.0
1         2        Christian Gobrecht  Unknown Nationality     1785.0     1844.0
2         3           Bela Lyon Pratt  Unknown Nationality     1867.0     1917.0
3         4            Mexican Artist  Unknown Nationality       -1.0       -1.0
4         5  James Gillinder and Sons             American     1861.0     1930.0
   ArtworkID                         Title Medium  YearStart  YearEnd          Nationality                 Artist  \
0          1  One-dollar Liberty Head Coin   Gold       1853     1853             American  James Barton Longacre   
1          2  Ten-dollar Liberty Head Coin   Gold       1901     1901  Unknown Nationality     Christian Gobrecht   
2          3    Two-and-a-Half Dollar Coin   Gold       1909     1927  Unknown Nationality         Unknown Artist   
3          4    Two-and-a-Half Dollar Coin   G

## processing moma_artists and moma_artworks

In [133]:
#Artist ID,Name,Nationality,Gender,Birth Year,Death Year
moma_artists = moma_artists[[
    "Artist ID",
    "Name",
    "Nationality",
    "Birth Year",
    "Death Year"
]].copy()

In [134]:
moma_artists = moma_artists.rename(columns={
    "Artist ID": "ArtistID",
    "Name": "Name",
    "Nationality": "Nationality",
    "Birth Year": "BirthYear",
    "Death Year": "DeathYear"
})

#replace null nationality
moma_artists["Nationality"] = moma_artists["Nationality"].fillna("Unknown Nationality")
#replace null birth year with -1
moma_artists["BirthYear"] = pd.to_numeric(moma_artists["BirthYear"], errors='coerce')
moma_artists["BirthYear"] = moma_artists["BirthYear"].fillna(-1)
#replace null death year with -1
moma_artists["DeathYear"] = pd.to_numeric(moma_artists["DeathYear"], errors='coerce')
moma_artists["DeathYear"] = moma_artists["DeathYear"].fillna(-1)

print(moma_artists.head())
print(moma_artists.isnull().sum())

   ArtistID             Name Nationality  BirthYear  DeathYear
0         1   Robert Arneson    American     1930.0     1992.0
1         2   Doroteo Arnaiz     Spanish     1936.0       -1.0
2         3      Bill Arnold    American     1941.0       -1.0
3         4  Charles Arnoldi    American     1946.0       -1.0
4         5      Per Arnoldi      Danish     1941.0       -1.0
ArtistID       0
Name           0
Nationality    0
BirthYear      0
DeathYear      0
dtype: int64


In [135]:
import re 
print(moma_art.columns.tolist())
moma_artworks = moma_art.loc[:,[
    "Artwork ID",
    "Title",
    "Medium",
    "Date",
    "Artist ID"
]].copy()

moma_artworks = moma_artworks.rename(columns={
    "Artwork ID": "ArtworkID",
    "Title": "Title",
    "Medium": "Medium",
    "Date": "Date",
    "Artist ID": "ArtistID"
})

def parse_simple_years(date_str):
    # parse first / second if exists strins of numbers
    if pd.isna(date_str):
        return (pd.NA, pd.NA)

    s = str(date_str)
    # normalize dashes just in case
    s = s.replace("–", "-").replace("—", "-")

    nums = re.findall(r"\d+", s)
    if not nums:
        return (pd.NA, pd.NA)

    # first chunk
    y1_str = nums[0]
    if len(y1_str) != 4:
        # exit if not 4 digit year
        return (pd.NA, pd.NA)
    y1 = int(y1_str)

    # only one chunk: start=end=y1
    if len(nums) == 1:
        return (y1, y1)

    # second chunk
    y2_str = nums[1]
    if len(y2_str) == 4:
        y2 = int(y2_str)
    elif len(y2_str) == 2:
        # yassume same centur
        century = y1 // 100
        y2 = century * 100 + int(y2_str)
    else:
        y2 = y1

    return (y1, y2)

# apply parser to DateRaw
year_pairs = moma_artworks["Date"].map(parse_simple_years)
moma_artworks["YearStart"], moma_artworks["YearEnd"] = zip(*year_pairs)

#replace null medium
moma_artworks["Medium"] = moma_artworks["Medium"].fillna("Unknown Medium")
#delete columns without title
moma_artworks = moma_artworks.dropna(subset=["Title"]).reset_index(drop=True)
#replace null artist id with -1
moma_artworks["ArtistID"] = pd.to_numeric(moma_artworks["ArtistID"], errors='coerce')
moma_artworks["ArtistID"] = moma_artworks["ArtistID"].fillna(-1)
#replace null yearstart with -1
moma_artworks["YearStart"] = pd.to_numeric(moma_artworks["YearStart"], errors='coerce')
moma_artworks["YearStart"] = moma_artworks["YearStart"].fillna(-1)
#add yearend column same as yearstart
moma_artworks["YearEnd"] = moma_artworks["YearStart"]
#add museum column with "Museum of Modern Art"
moma_artworks["Museum"] = "Museum of Modern Art"

#delete Date column
moma_artworks = moma_artworks.drop(columns=["Date"])

print(moma_artworks.head())
print(moma_artworks.isnull().sum())

['Artwork ID', 'Title', 'Artist ID', 'Name', 'Date', 'Medium', 'Dimensions', 'Acquisition Date', 'Credit', 'Catalogue', 'Department', 'Classification', 'Object Number', 'Diameter (cm)', 'Circumference (cm)', 'Height (cm)', 'Length (cm)', 'Width (cm)', 'Depth (cm)', 'Weight (kg)', 'Duration (s)']
   ArtworkID                                              Title                                             Medium  \
0          2  Ferdinandsbrücke Project, Vienna, Austria, Ele...      Ink and cut-and-pasted painted pages on paper   
1          3  City of Music, National Superior Conservatory ...                  Paint and colored pencil on print   
2          4  Villa near Vienna Project, Outside Vienna, Aus...  Graphite, pen, color pencil, ink, and gouache ...   
3          5  The Manhattan Transcripts Project, New York, N...  Photographic reproduction with colored synthet...   
4          6  Villa, project, outside Vienna, Austria, Exter...  Graphite, color pencil, ink, and gouache on tr..

# find possible duplicates between datasets

In [136]:
#save to csv
met_artists.to_csv("./data/met_artists_processed.csv", index=False)
met_artworks.to_csv("./data/met_artworks_processed.csv", index=False)
moma_artists.to_csv("./data/moma_artists_processed.csv", index=False)
moma_artworks.to_csv("./data/moma_artworks_processed.csv", index=False)

In [137]:
#find duplicate artists
duplicate_artists = pd.merge(
    met_artists,
    moma_artists,
    on=["Name", "BirthYear"],
    how="inner",
    suffixes=('_met', '_moma')
)
print("Duplicate Artists:")
print(duplicate_artists)

duplicate_artworks = pd.merge(
    met_artworks,
    moma_artworks,
    on=["Title", "YearStart"],
    how="inner",
    suffixes=('_met', '_moma')
)
print("Duplicate Artworks:")
print(duplicate_artworks)

Duplicate Artists:
      ArtistID_met                   Name      Nationality_met  BirthYear  DeathYear_met  ArtistID_moma  \
0               21  Louis Comfort Tiffany             American     1848.0         1933.0           5876   
1               45     Frank Lloyd Wright             American     1867.0         1959.0           6459   
2              283           Samuel Gragg  Unknown Nationality     1772.0         1855.0           2280   
3              297        Wallace Nutting             American     1861.0         1941.0           4349   
4              319        Gustav Stickley             American     1858.0         1942.0           7077   
...            ...                    ...                  ...        ...            ...            ...   
1816         55882             Otto Eglau  Unknown Nationality     1917.0         1988.0           1691   
1817         55897          Anthony Gross  Unknown Nationality     1905.0         1984.0           2362   
1818         55910

In [138]:
print("duplicate_artists columns:", duplicate_artists.columns.tolist())
print("duplicate_artworks columns:", duplicate_artworks.columns.tolist())
print("moma_artworks columns:", moma_artworks.columns.tolist())

#omit duplicate artworks from moma_artworks
moma_artworks_filtered = moma_artworks[~moma_artworks["ArtworkID"].isin(duplicate_artworks["ArtworkID_moma"])].reset_index(drop=True)

#before deleting duplicate artists, map artist ids in moma_artworks_filtered to met_artists
artist_id_map_moma_to_met = dict(zip(duplicate_artists["ArtistID_moma"], duplicate_artists["ArtistID_met"]))
moma_artworks_filtered["ArtistID"] = moma_artworks_filtered["ArtistID"].map(artist_id_map_moma_to_met).fillna(moma_artworks_filtered["ArtistID"]).astype(int)
print(moma_artworks_filtered.head())

#delete duplicate artists from moma_artists
moma_artists_filtered = moma_artists[~moma_artists["ArtistID"].isin(duplicate_artists["ArtistID_moma"])].reset_index(drop=True)

#produce final datasets
moma_artists = moma_artists_filtered
moma_artworks = moma_artworks_filtered
met_artists = met_artists
met_artworks = met_artworks

duplicate_artists columns: ['ArtistID_met', 'Name', 'Nationality_met', 'BirthYear', 'DeathYear_met', 'ArtistID_moma', 'Nationality_moma', 'DeathYear_moma']
duplicate_artworks columns: ['ArtworkID_met', 'Title', 'Medium_met', 'YearStart', 'YearEnd_met', 'Nationality', 'Artist', 'ArtistID_met', 'Museum_met', 'ArtworkID_moma', 'Medium_moma', 'ArtistID_moma', 'YearEnd_moma', 'Museum_moma']
moma_artworks columns: ['ArtworkID', 'Title', 'Medium', 'ArtistID', 'YearStart', 'YearEnd', 'Museum']
   ArtworkID                                              Title                                             Medium  \
0          2  Ferdinandsbrücke Project, Vienna, Austria, Ele...      Ink and cut-and-pasted painted pages on paper   
1          3  City of Music, National Superior Conservatory ...                  Paint and colored pencil on print   
2          4  Villa near Vienna Project, Outside Vienna, Aus...  Graphite, pen, color pencil, ink, and gouache ...   
3          5  The Manhattan Transcrip

In [139]:
#combine into artists and artworks
artists = pd.concat([met_artists, moma_artists], ignore_index=True)
artworks = pd.concat([met_artworks, moma_artworks], ignore_index=True)

#check for nulls and duplicates
print("Artists nulls:")
print(artists.isnull().sum())
print("Artists duplicates:")
print(artists.duplicated(subset=["Name", "BirthYear"]).sum())
print("Artworks nulls:")
print(artworks.isnull().sum())
print("Artworks duplicates:")
print(artworks.duplicated(subset=["Title", "YearStart"]).sum())

Artists nulls:
ArtistID       0
Name           0
Nationality    0
BirthYear      0
DeathYear      0
dtype: int64
Artists duplicates:
2
Artworks nulls:
ArtworkID           0
Title               0
Medium              0
YearStart           0
YearEnd             0
Nationality    122123
Artist         122123
ArtistID            0
Museum              0
dtype: int64
Artworks duplicates:
167661


In [140]:
artworks.loc[artworks["ArtistID"] == -1, "Artist"] = "Unknown Artist"

## export to csv

In [141]:
#check for duplicate ID
print("Artists duplicate IDs:")
print(artists.duplicated(subset=["ArtistID"]).sum())
print("Artworks duplicate IDs:")
print(artworks.duplicated(subset=["ArtworkID"]).sum())

Artists duplicate IDs:
13200
Artworks duplicate IDs:
54899


In [142]:
#rename artworkID to row number
artworks = artworks.reset_index(drop=True)
artworks["ArtworkID"] = artworks.index + 1

In [143]:
#keep old id's and reassign sequential IDs to artists
artists = artists.reset_index(drop=True)
artists["OldArtistID"] = artists["ArtistID"]

artists["ArtistID"] = artists.index + 1

# old to new id map
id_map_old_to_new = dict(zip(artists["OldArtistID"], artists["ArtistID"]))

# update artworks
artworks["ArtistID"] = artworks["ArtistID"].map(id_map_old_to_new).fillna(-1).astype(int)


In [145]:
#if no nationality, check if artist exists in other dataset
for index, row in artworks.iterrows():
    if row["Nationality"] == None:
        artist_id = row["ArtistID"]
        if artist_id != -1:
            #check in artists dataset
            artist_row = artists[artists["ArtistID"] == artist_id]
            if not artist_row.empty:
                nationality = artist_row["Nationality"].values[0]
                artworks.at[index, "Nationality"] = nationality

artworks["Nationality"] = artworks["Nationality"].fillna("Unknown Nationality")
print(artworks.isnull().sum())

ArtworkID           0
Title               0
Medium              0
YearStart           0
YearEnd             0
Nationality         0
Artist         114527
ArtistID            0
Museum              0
dtype: int64


In [148]:
#replace null artist with "Unknown Artist"
artworks["Artist"] = artworks["Artist"].fillna("Unknown Artist")

In [150]:
artists.to_csv("./data/artists_combined.csv", index=False)
artworks.to_csv("./data/artworks_combined.csv", index=False)